In [9]:
# Importing useful packages.
import numpy as np
import pandas as pd
import os
import bz2
import json
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse
from random import randrange
from bertopic import BERTopic
from sklearn.preprocessing import OneHotEncoder
import re
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Importing our utilitary modules.
import src.utils as utils
import src.feature_extraction as feature_extraction
import src.plot as plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Defining constants.
DATA_DIR = 'Data'
CACHE_DIR = 'Cache'
SPEAKER_INFO_FILE_PATH = os.path.join(DATA_DIR, 'speaker_attributes.parquet')
PREPROCESSED_DATASET_FILE_PATH = os.path.join(CACHE_DIR, "preprocessed_dataset.json.bz2")

In [11]:
@utils.cache_to_file_pickle("function-make_features_labels_arrays", cache_dir = CACHE_DIR, ignore_kwargs = ['batch_size'])
def make_features_labels_arrays(preprocessed_dataset_file_path, batch_size = 1000000):
    
    def process_batch(features, quotation_topic, num_occurrences,
                      features_batch, quotations_batch, num_occurrences_batch):
                
        # If list is empty, don't do anything otherwise BERTopic raises un exception.
        if features_batch:
            topics, _ = bert_model.transform(quotations_batch)

            # Convert sparse list of lists to COO sparse matrix. 
            # COO sparse matrices have the advantage of being quick to generate and to stack.
            features.append( scipy.sparse.coo_matrix(features_batch) )

            # These can simply be added at the end of the list as they are not sparse.
            quotation_topic.extend( topics )
            num_occurrences.extend( num_occurrences_batch )

        return features, quotation_topic, num_occurrences
    
    features_cols_titles = None
    
    features = []
    quotation_topic = []
    num_occurrences = []
    
    features_batch = []
    quotations_batch = []
    num_occurrences_batch = []
        
    for line in utils.json_lines_generator(preprocessed_dataset_file_path):        
        num_occurrences_batch.append(line.pop('num_occurrences'))
        quotations_batch.append(line.pop('quotation'))
        
        if features_cols_titles is None:
            features_cols_titles = sorted(line)

        features_batch.append([line[key] for key in features_cols_titles])
        
        if len(features_batch) >= batch_size:
            features, quotation_topic, num_occurrences = process_batch(features, quotation_topic, num_occurrences,
                                                                       features_batch, quotations_batch, num_occurrences_batch)
            features_batch = []
            quotations_batch = []
            num_occurrences_batch = []
            
    features, quotation_topic, num_occurrences = process_batch(features, quotation_topic, num_occurrences,
                                                               features_batch, quotations_batch, num_occurrences_batch)

    # Merge list of sparse matrices into a single sparse matrix.
    # Here COO sparse format is useful because this stacking is done faster than with other formats.
    features = scipy.sparse.vstack(features)
    
    # Convert list to a single numpy array.
    quotation_topic = np.array(quotation_topic)
    num_occurrences = np.array(num_occurrences)
        
    # One-hot encode quotation topics.
    topic_number_names_map = bert_model.get_topic_info().set_index('Topic')
    topic_number_names_map.at[-1, 'Name'] = 'unknown'

    quotation_topic_names = topic_number_names_map.loc[quotation_topic, 'Name'].values
    
    encoder = OneHotEncoder(sparse = True, dtype = 'int')
    one_hot_quotation_topics = encoder.fit_transform(np.reshape(quotation_topic_names, (-1, 1)))
    
    # Add quotation topics as columns of feature matrix.
    features = scipy.sparse.hstack([features, one_hot_quotation_topics])
    encoder_categories, = encoder.categories_
    features_cols_titles.extend([f'quotation_topic_{col_name.upper()}' for col_name in encoder_categories])
     
    # Convert sparse matrix from COO to CSR format for easy slicing later on.
    features = features.tocsr()
    
    return features, num_occurrences, features_cols_titles

## 3.B.a Big Function Doing all the stuff needed for one full run with a soecific viral_thr

In [12]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.svm import LinearSVC
from imblearn.combine import SMOTETomek

from IPython.display import display

In [30]:
def do_train_test_split(VIRAL_THR):
    # Load inside function to actually save RAM when deleting them.
    features, num_occurrences, features_cols_titles = make_features_labels_arrays(preprocessed_dataset_file_path = PREPROCESSED_DATASET_FILE_PATH)

    # Train-Test stratified split with test ratio of 0.3
    viral_label = np.int32(num_occurrences > VIRAL_THR)

    print(f"Viral Thr: {VIRAL_THR}, {viral_label.mean():.3%} quotes are viral")    

    splits = train_test_split(features, num_occurrences, viral_label,
                              test_size = 0.3,
                              shuffle = True,
                              stratify = viral_label)

    return splits


def sweep_l1_regularization(features, labels, cv_n_splits = 10, downsampling_factor = 1,
                            resample_balanced = False, n_jobs = 4,
                            l1_log_min = -5, l1_log_max = 2, l1_log_steps = 50):

    if downsampling_factor != 1:
        # Use train_test_split to downsample dataset in a stratified manner.
        _, features, _, labels = train_test_split(features, labels,
                                                  test_size = 1. / downsampling_factor,
                                                  shuffle = True,
                                                  stratify = labels)
        
    if resample_balanced:
        features, labels = SMOTETomek().fit_resample(features, labels)
    
    print(f"Training on {len(labels)} samples with {features.shape[1]} features")
    print(f"Positive samples: {np.sum(labels == 1)}, negative samples: {np.sum(labels == 0)}")

    res = GridSearchCV(estimator  = LinearSVC(dual = False, max_iter = 10000, class_weight = 'balanced'), 
                       param_grid = {'penalty': ('l1', ), 'C': np.logspace(l1_log_min, l1_log_max, l1_log_steps)},
                       cv         = StratifiedKFold(cv_n_splits),
                       scoring    = {'accuracy' : 'accuracy',
                                     'precision': 'precision',
                                     'recall'   : 'recall',
                                     'f1'       : 'f1',
                                     'sparsity' : lambda estimator, X, y: np.mean(np.abs(estimator.coef_) < 1e-8)}, 
                       return_train_score = True,
                       refit = False,
                       n_jobs = n_jobs,
                       verbose = 2).fit(features, labels)

    df = pd.DataFrame(res.cv_results_)
    df = df[[col for col in df.columns if col.startswith('param_') or re.match(r'^(mean|std)_(train|test)_', col)]]

    # Changing names of sparsity columns and dropping duplicates.
    df = df.drop(columns = ['mean_test_sparsity', 'std_test_sparsity']).rename(columns = {'mean_train_sparsity': 'mean_sparsity',
                                                                                          'std_train_sparsity' : 'std_sparsity'})
    return df






def sweep_l1_regularization_this_one_is_old_forces_class_balance_which_works_not_well_in_test_set_unfortunately_use_other \
    (features, labels, cv_n_splits = 10, downsampling_factor = 'lowest', l1_log_min = -5, l1_log_max = 2, l1_log_steps = 50):

    allowed_print_warning = True
    if downsampling_factor == 'lowest':
        allowed_print_warning = False
        downsampling_factor = 1
    
    # Estimate number of samples of each class needed.
    num_samples_per_class = len(labels) // (2 * downsampling_factor)
    
    # Get number of samples available for smaller class.
    num_positive_samples = np.sum(labels)    
    num_samples_smaller_class = min(num_positive_samples, len(labels) - num_positive_samples)    
    
    if allowed_print_warning and num_samples_smaller_class < num_samples_per_class:
        print(f"[WARNING]: Requested downsampling_factor of {downsampling_factor} with {len(labels)} samples. " + \
              f"This would require {num_samples_per_class} samples per class, but minority class only has " + \
              f"{num_samples_smaller_class} samples. Using {2 * num_samples_smaller_class} samples to train per " + \
              f"class instead.\n\n")
    
    # Split training samples into balanced classes.    
    idx_positive_samples, = np.where(labels == 1)
    idx_negative_samples, = np.where(labels == 0)
    
    idx_positive_samples = np.random.choice(idx_positive_samples, size = num_samples_smaller_class, replace = False)
    idx_negative_samples = np.random.choice(idx_negative_samples, size = num_samples_smaller_class, replace = False)

    all_idx = list(idx_positive_samples) + list(idx_negative_samples)
    
    features = features[all_idx]
    labels   = labels  [all_idx]
    
    print(f"Training on {len(labels)} samples with {features.shape[1]} features")
    
    res = GridSearchCV(estimator  = LinearSVC(dual = False, max_iter = 10000), 
                       param_grid = {'penalty': ('l1', ), 'C': np.logspace(l1_log_min, l1_log_max, l1_log_steps)},
                       cv         = StratifiedKFold(cv_n_splits),
                       scoring    = {'accuracy' : 'accuracy',
                                     'precision': 'precision',
                                     'recall'   : 'recall',
                                     'f1'       : 'f1',
                                     'sparsity' : lambda estimator, X, y: np.mean(np.abs(estimator.coef_) < 1e-8)}, 
                       return_train_score = True,
                       refit = False,
                       n_jobs = 8,
                       verbose = 2).fit(features, labels)

    df = pd.DataFrame(res.cv_results_)
    df = df[[col for col in df.columns if col.startswith('param_') or re.match(r'^(mean|std)_(train|test)_', col)]]

    # Changing names of sparsity columns and dropping duplicates.
    df = df.drop(columns = ['mean_test_sparsity', 'std_test_sparsity']).rename(columns = {'mean_train_sparsity': 'mean_sparsity',
                                                                                          'std_train_sparsity' : 'std_sparsity'})
    return df, all_idx


def plot_columns_over_column(df, x_axis_column, y_axis_columns, x_log_scale = True, figsize = (10, 7)):    
    fig, ax = plt.subplots(1, 1, figsize = figsize)
    
    if x_log_scale:
        ax.set(xscale = 'log')
    
    for y in y_axis_columns:        
        ax.errorbar(df[x_axis_column], df[y], df[y.replace('mean', 'std')], label = y.replace('_', ' ').title())
        
    plt.xlabel(x_axis_column.replace('_', ' ').title())
    plt.legend()
    plt.show()

In [31]:
def big_func(VIRAL_THR, l1_log_min = -5, l1_log_max = 0, resample_balanced = False):
    # Seed numpy for reproducibility of all following code.
    np.random.seed(0x616461)
    
    features_train, features_test, num_occurrences_train, num_occurrences_test, viral_label_train, viral_label_test = do_train_test_split(VIRAL_THR)
    
    df = sweep_l1_regularization(features_train, viral_label_train, 
                                 cv_n_splits = 10, # Can be reduced down to a minimum of 5 if needed
                                 downsampling_factor = 10, # How much to downsample the data. Note that downsampling is stratified,
                                                          # so for large values of viral_thr each fold may have very few points.
                                                          # This is exactly the issue I had, and why we need to lower viral_thr,
                                                          # otherwise models just won't learn.  
                                                          # Larger values will make it faster to run, but may lead to worse performance.
                                 n_jobs = 8, # This one depends on your CPU and RAM. Higher is faster, but only if you don't fill
                                              # your RAM and your CPU is not at 100% all cores. 
                                              # Start with one, using a fixed downsampling factor and increase it until RAM is filled,
                                              # up to a maximum the number of physical cores on your machine.
                                              # /!\ THE COMBINATION OF HIGH N_JOBS AND LOW DOWNSAMPLING FACTOR CAN CRASH YOUR SYSTEM!
                                              # (TECHNICALLY ONLY MAKE IT EXTREMELY UNRESPONSIVE, STILL NOT NICE)
                                 l1_log_min = l1_log_min, l1_log_max = l1_log_min, l1_log_steps = 20, 
                                              # Min max and number os steps of sweep logspace.
                                              # Min and max probably do not need to be changed.
                                              # steps may be changed. Lower makes run faster,
                                              # but also decreases resolution of plots.
                                              # Warning: due to how regularized SVM 
                                              # works, large values of C parameter may
                                              # lead to very slow convergence (I can provide
                                              # intuitive explaination if needed). So for time
                                              # reasons it may be intersting to decrease l1_log_max,
                                              # FOR AS LONG AS A CONVERGENCE TO A MAX VAL F1 SCORE
                                              # CAN STILL BE OBSERVED IN THE PLOTS!!!
                                 resample_balanced = resample_balanced) # This one is worth trying: using it for viral_thr
                                              # which are deeply unbalanced should allow to achieve better performance.
    
    # You can see how far in the cv you are in the jupyter console (the black terminal that is opened when you lanch jupyter).
    # For some reason, the logs from sklearn CV are written there when n_jobs > 1.
    # You can ignore all the UndefinedMetricWarning, unfortunately I was not able to tell sklearn not to show them.
    
    
    pd.options.display.max_rows = 1000
    display(df)
    
    plot_columns_over_column(df, 'param_C', [col for col in df.columns if col.startswith('mean_test_')] + ['mean_sparsity'])

# AIM: FIND A VIRAL_THR WHICH TRAINS WELL (GOOD VAL F1 SCORE), ALLOWS TO HAVE GOOD SPARSITY (0.1 SPARSITY WITH NO SIGNIFICANT DROP IN VAL F1 SCORE), AND HAS APPROX <20% OF DATASET WHICH IS A VIRAL QUOTE
The importance of each of these parameters can be discussed.

In [21]:
big_func(VIRAL_THR = 100, resample_balanced = False, l1_log_min = -5, l1_log_max = -3)

Viral Thr: 100, 0.399% quotes are viral


In [ ]:
big_func(VIRAL_THR = 100, resample_balanced = True, l1_log_min = -5, l1_log_max = -3)

In [ ]:
big_func(VIRAL_THR = 5)

Viral Thr: 5, 9.330% quotes are viral
Training on 4834528 samples with 613 features
Positive samples: 451047, negative samples: 4383481
Fitting 10 folds for each of 40 candidates, totalling 400 fits


In [ ]:
clf = LinearSVC(dual = False, max_iter = 10000, penalty = 'l1', C = 0.01).fit(features_train[all_idx], viral_label_train[all_idx])

In [32]:
pred = clf.predict(features_test)

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

f1_score(viral_label_test, pred)

0.03330606282527229

<a id='training_regression'></a>
# 4. Regressing number of occurences

<a id='theory_reg'></a>
## 4.A. Theoretical explanations of the methods used to perform the regression
[Back to table of content](#table_of_contents)

In this section, we will describe the mathematical models/concepts of the machine learning methods we will use in order to regress the occurrence of a quote from some features previously selected. 

We will focus on a simple regression method: linear regression.

<a id='lin_reg_theory'></a>
### 4.A.a. Linear Regression

Regression problems are problems aiming at finding a function $f$ that estimates the relationship between categorical or continuous inputs and an output continuous variable.

Linear regression is a specific modelling technique to resolve regression problems. Specifically, it is a model in which we assume that the output variable can be predicted from a linear combination of the input variables such that: 

$$Y = f(X) = \alpha_0 + x_1\alpha_1 ... + x_n\alpha_n$$

Training of the model aims to find the coefficients $\alpha_i$. To find these, we typically minimize the mean squared error (MSE) defined as follows:

$$L_{MSE}(\alpha; x_i, y_i) = (y_i - f(\alpha; x_i))^2$$

We will also try to add $L2$ or $L1$ regularization terms to push the less meaningful terms towards 0 which will improve the interpretability of the results. In case of the linear regression with no regularization or with $L2$ regularization the solution has a closed-form. For $L1$ regularization there is no closed-form solution. When a closed-form solution is not available, linear regression can be solved via gradient descent. Even if a closed-form solution is available, it should be noted that it may require large amounts of memory to be computed for large training sets.

We want to start with linear regression for multiple reasons. The first one is that it is a simple, easy-to-train model which will allow us to handle bigger subsets of the dataset for training in a reasonable time. It's also interesting to see if linear relationships are enough to achieve rather good accuracy for such a complex dataset and it will constitute a good indicator as to whether low bias can be achieved with such a low complexity model.

For this model to be acceptable, there are some assumptions:
- The relationship between $X$ and $Y$ (features and quote count popularity) are linearly related. This might not be the case but we want to see if it is by gauging the accuracy of the model.

- The observations are independent of each other. Of course, this can never truly be the case in such a huge dataset influenced by extremely popular extremely quoted individuals, real-world events, and a ton of other params but we still think that we can apply this model with some amount of success.